In [1]:
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
train_df = pd.read_csv("../../input/feedback-prize-english-language-learning/train.csv")
train_df.head(2)

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5


In [3]:
model_path_list = [
    #('../../14_Baseline4/exp/result/14_v1_01/oof_df.csv',1), # deberta-v3-base
    #('../../14_Baseline4/exp/result/14_v1_07/oof_df.csv',1), # deberta-v3-base, 10folds    
    #('../../14_Baseline4/exp/result/14_v1_08/oof_df.csv',1), # deberta-base
    #('../../14_Baseline4/exp/result/14_v1_09/oof_df.csv',1), # roberta-base
    #('../../14_Baseline4/exp/result/14_v1_10/oof_df.csv',1), # deberta-v3-large
    #('../../14_Baseline4/exp/result/14_v1_11/oof_df.csv',1), # xlm-roberta-base
    #('../../14_Baseline4/exp/result/14_v1_12/oof_df.csv',1), # deberta-large
    #('../../14_Baseline4/exp/result/14_v1_13/oof_df.csv',1), # roberta-large
    #('../../14_Baseline4/exp/result/14_v1_14/oof_df.csv',1), # xlm-roberta-large
#     ('../../14_Baseline4/exp/result/14_v1_15/oof_df.csv',1), # deberta-v2-xlarge
#     ('../../14_Baseline4/exp/result/14_v1_16/oof_df.csv',1), # deberta-xlarge
    
#     ('../../14_Baseline4/exp/result/14_v2_01/oof_df.csv',1), # deberta-v3-base, seed200
#     ('../../14_Baseline4/exp/result/14_v2_02/oof_df.csv',1), # deberta-v3-large, seed200
#     ('../../14_Baseline4/exp/result/14_v2_03/oof_df.csv',1), # deberta-base, seed200
    
    #('../../17_Pseudo3/exp/result/17_v1_01/oof_df.csv',1), # deberta-v3-base
    ('../../17_Pseudo3/exp/result/17_v1_02/oof_df.csv',1), # deberta-v3-base
    ('../../17_Pseudo3/exp/result/17_v1_03/oof_df.csv',1), # deberta-v3-large
    #('../../17_Pseudo3/exp/result/17_v1_04/oof_df.csv',1), # deberta-v3-base, seed200
    ('../../17_Pseudo3/exp/result/17_v1_05/oof_df.csv',1), # deberta-base
    ('../../17_Pseudo3/exp/result/17_v1_06/oof_df.csv',1), # deberta-large
    ('../../17_Pseudo3/exp/result/17_v1_07/oof_df.csv',1), # deberta-v3-small
    
    #('../../18_Pseudo4/exp/result/18_v1_01/oof_df.csv',1), # deberta-v3-base
    #('../../18_Pseudo4/exp/result/18_v1_02/oof_df.csv',1), # deberta-v3-large
    #('../../18_Pseudo4/exp/result/18_v1_03/oof_df.csv',1), # deberta-v3-small
]

oof_df_list = [
    pd.read_csv(model_path) for model_path,_ in model_path_list
]
weights_list = [
    w for _,w in model_path_list
]
weights_list = [w/sum(weights_list) for w in weights_list]

for oof_df in oof_df_list:
    oof_df = train_df[['text_id']].merge(oof_df, how='left', on='text_id')
    
num_models = len(model_path_list)

TARGET_COLS = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']

preds = 0
for oof_df, w in zip(oof_df_list, weights_list):
    preds += oof_df[TARGET_COLS].values * w

oofs = []
for oof_df in oof_df_list:
    oofs.append(oof_df[TARGET_COLS].values)

In [4]:
oof_df = oof_df_list[0].copy()
for i,col in enumerate(TARGET_COLS):
    oof_df[col] = preds[:,i]

In [5]:
import numpy as np

def calc_metric(pred, gt):
    '''
    pred : (num_data, num_labels)
    gt : (num_data, num_labels)
    '''
    score = np.sqrt(np.mean((pred - gt)**2, axis=0))
    score = score.mean()
    return score

In [6]:
score = calc_metric(pred=oof_df[TARGET_COLS].values, gt=train_df[TARGET_COLS].values)
print('CV={:.4f}'.format(score))

CV=0.4458


# Optimize class-wise

In [7]:
ensemble_predictions=np.stack(oofs)
ensemble_predictions.shape

(5, 3911, 6)

In [8]:
from skopt import gp_minimize

def run_optimize(i):
    target_col = TARGET_COLS[i]
    print("*"*50)
    print(target_col)
    print("*"*50)
    def ensemble_score(weights,return_pred=False):
        weights=np.array(weights)
        weights=weights.reshape(-1,1,1)/weights.sum()
        p=weights.reshape(-1,1,1)*ensemble_predictions[:,:,i,None]
        p=p.sum(0)
        score=calc_metric(p, train_df[[target_col]])
        if return_pred:
            return score,p
        else:
            return score   
    results = gp_minimize(ensemble_score, np.array([[0.1,1] for i in range(len(ensemble_predictions))]),
                          verbose=True,random_state=2022)
    best_weights=np.array(results['x'])/sum(results['x'])
    print(best_weights)
    # smaller is better for this metric
    score, ensemble_pred=ensemble_score(best_weights,True)
    print("score={:.4f}", score)
    return score, best_weights

In [9]:
scores = {}
best_weights = {}
for i,target_col in enumerate(TARGET_COLS):
    score, weights = run_optimize(i)
    scores[target_col] = score
    best_weights[target_col] = weights

**************************************************
cohesion
**************************************************
Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0031
Function value obtained: 0.4744
Current minimum: 0.4744
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0019
Function value obtained: 0.4754
Current minimum: 0.4744
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0019
Function value obtained: 0.4748
Current minimum: 0.4744
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.0019
Function value obtained: 0.4756
Current minimum: 0.4744
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.

Iteration No: 41 ended. Search finished for the next optimal point.
Time taken: 0.6042
Function value obtained: 0.4752
Current minimum: 0.4737
Iteration No: 42 started. Searching for the next optimal point.
Iteration No: 42 ended. Search finished for the next optimal point.
Time taken: 0.6314
Function value obtained: 0.4737
Current minimum: 0.4737
Iteration No: 43 started. Searching for the next optimal point.
Iteration No: 43 ended. Search finished for the next optimal point.
Time taken: 0.6354
Function value obtained: 0.4745
Current minimum: 0.4737
Iteration No: 44 started. Searching for the next optimal point.
Iteration No: 44 ended. Search finished for the next optimal point.
Time taken: 0.6286
Function value obtained: 0.4751
Current minimum: 0.4737
Iteration No: 45 started. Searching for the next optimal point.
Iteration No: 45 ended. Search finished for the next optimal point.
Time taken: 0.5791
Function value obtained: 0.4737
Current minimum: 0.4737
Iteration No: 46 started. Sea

Iteration No: 81 ended. Search finished for the next optimal point.
Time taken: 0.9073
Function value obtained: 0.4737
Current minimum: 0.4737
Iteration No: 82 started. Searching for the next optimal point.
Iteration No: 82 ended. Search finished for the next optimal point.
Time taken: 0.9770
Function value obtained: 0.4741
Current minimum: 0.4737
Iteration No: 83 started. Searching for the next optimal point.
Iteration No: 83 ended. Search finished for the next optimal point.
Time taken: 0.9181
Function value obtained: 0.4760
Current minimum: 0.4737
Iteration No: 84 started. Searching for the next optimal point.
Iteration No: 84 ended. Search finished for the next optimal point.
Time taken: 0.9472
Function value obtained: 0.4749
Current minimum: 0.4737
Iteration No: 85 started. Searching for the next optimal point.
Iteration No: 85 ended. Search finished for the next optimal point.
Time taken: 0.9514
Function value obtained: 0.4737
Current minimum: 0.4737
Iteration No: 86 started. Sea

Iteration No: 21 ended. Search finished for the next optimal point.
Time taken: 0.3747
Function value obtained: 0.4403
Current minimum: 0.4392
Iteration No: 22 started. Searching for the next optimal point.
Iteration No: 22 ended. Search finished for the next optimal point.
Time taken: 0.3374
Function value obtained: 0.4392
Current minimum: 0.4392
Iteration No: 23 started. Searching for the next optimal point.
Iteration No: 23 ended. Search finished for the next optimal point.
Time taken: 0.3810
Function value obtained: 0.4398
Current minimum: 0.4392
Iteration No: 24 started. Searching for the next optimal point.
Iteration No: 24 ended. Search finished for the next optimal point.
Time taken: 0.3213
Function value obtained: 0.4396
Current minimum: 0.4392
Iteration No: 25 started. Searching for the next optimal point.
Iteration No: 25 ended. Search finished for the next optimal point.
Time taken: 0.3550
Function value obtained: 0.4392
Current minimum: 0.4392
Iteration No: 26 started. Sea

Iteration No: 61 ended. Search finished for the next optimal point.
Time taken: 0.6829
Function value obtained: 0.4401
Current minimum: 0.4392
Iteration No: 62 started. Searching for the next optimal point.
Iteration No: 62 ended. Search finished for the next optimal point.
Time taken: 0.7541
Function value obtained: 0.4392
Current minimum: 0.4392
Iteration No: 63 started. Searching for the next optimal point.
Iteration No: 63 ended. Search finished for the next optimal point.
Time taken: 0.7049
Function value obtained: 0.4392
Current minimum: 0.4392
Iteration No: 64 started. Searching for the next optimal point.
Iteration No: 64 ended. Search finished for the next optimal point.
Time taken: 0.8048
Function value obtained: 0.4401
Current minimum: 0.4392
Iteration No: 65 started. Searching for the next optimal point.
Iteration No: 65 ended. Search finished for the next optimal point.
Time taken: 0.7022
Function value obtained: 0.4392
Current minimum: 0.4392
Iteration No: 66 started. Sea

...

Iteration No: 10 ended. Evaluation done at random point.
Time taken: 0.3218
Function value obtained: 0.4651
Current minimum: 0.4641
Iteration No: 11 started. Searching for the next optimal point.
Iteration No: 11 ended. Search finished for the next optimal point.
Time taken: 0.3225
Function value obtained: 0.4652
Current minimum: 0.4641
Iteration No: 12 started. Searching for the next optimal point.
Iteration No: 12 ended. Search finished for the next optimal point.
Time taken: 0.3307
Function value obtained: 0.4652
Current minimum: 0.4641
Iteration No: 13 started. Searching for the next optimal point.
Iteration No: 13 ended. Search finished for the next optimal point.
Time taken: 0.3866
Function value obtained: 0.4634
Current minimum: 0.4634
Iteration No: 14 started. Searching for the next optimal point.
Iteration No: 14 ended. Search finished for the next optimal point.
Time taken: 0.3357
Function value obtained: 0.4656
Current minimum: 0.4634
Iteration No: 15 started. Searching for 

Iteration No: 50 ended. Search finished for the next optimal point.
Time taken: 0.6275
Function value obtained: 0.4662
Current minimum: 0.4632
Iteration No: 51 started. Searching for the next optimal point.
Iteration No: 51 ended. Search finished for the next optimal point.
Time taken: 0.5973
Function value obtained: 0.4653
Current minimum: 0.4632
Iteration No: 52 started. Searching for the next optimal point.
Iteration No: 52 ended. Search finished for the next optimal point.
Time taken: 0.6176
Function value obtained: 0.4663
Current minimum: 0.4632
Iteration No: 53 started. Searching for the next optimal point.
Iteration No: 53 ended. Search finished for the next optimal point.
Time taken: 0.5933
Function value obtained: 0.4654
Current minimum: 0.4632
Iteration No: 54 started. Searching for the next optimal point.
Iteration No: 54 ended. Search finished for the next optimal point.
Time taken: 0.6764
Function value obtained: 0.4632
Current minimum: 0.4632
Iteration No: 55 started. Sea

Iteration No: 90 ended. Search finished for the next optimal point.
Time taken: 0.9259
Function value obtained: 0.4632
Current minimum: 0.4632
Iteration No: 91 started. Searching for the next optimal point.
Iteration No: 91 ended. Search finished for the next optimal point.
Time taken: 1.0061
Function value obtained: 0.4653
Current minimum: 0.4632
Iteration No: 92 started. Searching for the next optimal point.
Iteration No: 92 ended. Search finished for the next optimal point.
Time taken: 1.0880
Function value obtained: 0.4652
Current minimum: 0.4632
Iteration No: 93 started. Searching for the next optimal point.
Iteration No: 93 ended. Search finished for the next optimal point.
Time taken: 1.1735
Function value obtained: 0.4648
Current minimum: 0.4632
Iteration No: 94 started. Searching for the next optimal point.
Iteration No: 94 ended. Search finished for the next optimal point.
Time taken: 1.1411
Function value obtained: 0.4632
Current minimum: 0.4632
Iteration No: 95 started. Sea

Iteration No: 30 ended. Search finished for the next optimal point.
Time taken: 0.3854
Function value obtained: 0.4395
Current minimum: 0.4373
Iteration No: 31 started. Searching for the next optimal point.
Iteration No: 31 ended. Search finished for the next optimal point.
Time taken: 0.3478
Function value obtained: 0.4374
Current minimum: 0.4373
Iteration No: 32 started. Searching for the next optimal point.
Iteration No: 32 ended. Search finished for the next optimal point.
Time taken: 0.4406
Function value obtained: 0.4383
Current minimum: 0.4373
Iteration No: 33 started. Searching for the next optimal point.
Iteration No: 33 ended. Search finished for the next optimal point.
Time taken: 0.5723
Function value obtained: 0.4378
Current minimum: 0.4373
Iteration No: 34 started. Searching for the next optimal point.
Iteration No: 34 ended. Search finished for the next optimal point.
Time taken: 0.5523
Function value obtained: 0.4397
Current minimum: 0.4373
Iteration No: 35 started. Sea

Iteration No: 70 ended. Search finished for the next optimal point.
Time taken: 0.8061
Function value obtained: 0.4388
Current minimum: 0.4373
Iteration No: 71 started. Searching for the next optimal point.
Iteration No: 71 ended. Search finished for the next optimal point.
Time taken: 0.8266
Function value obtained: 0.4373
Current minimum: 0.4373
Iteration No: 72 started. Searching for the next optimal point.
Iteration No: 72 ended. Search finished for the next optimal point.
Time taken: 0.7809
Function value obtained: 0.4382
Current minimum: 0.4373
Iteration No: 73 started. Searching for the next optimal point.
Iteration No: 73 ended. Search finished for the next optimal point.
Time taken: 0.8261
Function value obtained: 0.4384
Current minimum: 0.4373
Iteration No: 74 started. Searching for the next optimal point.
Iteration No: 74 ended. Search finished for the next optimal point.
Time taken: 0.8484
Function value obtained: 0.4393
Current minimum: 0.4373
Iteration No: 75 started. Sea

In [10]:
best_weights

{'cohesion': array([0.17728539, 0.42992961, 0.04299296, 0.26168512, 0.08810692]),
 'syntax': array([0.40504445, 0.40504445, 0.04050445, 0.1089022 , 0.04050445]),
 'vocabulary': array([0.38263708, 0.38263708, 0.03826371, 0.15819843, 0.03826371]),
 'phraseology': array([0.36652667, 0.36652667, 0.03665267, 0.19364133, 0.03665267]),
 'grammar': array([0.2837088 , 0.55099323, 0.05509932, 0.05509932, 0.05509932]),
 'conventions': array([0.47379813, 0.38406244, 0.04737981, 0.04737981, 0.04737981])}

In [11]:
scores

{'cohesion': 0.47367978526729315,
 'syntax': 0.43921055880768667,
 'vocabulary': 0.40646109933437435,
 'phraseology': 0.4485689601481223,
 'grammar': 0.4632334327472205,
 'conventions': 0.437329916864718}

In [12]:
CV = sum(scores.values()) / len(scores)
print("CV={:.4f}".format(CV))

CV=0.4447


# Error Analysis - Check Corr

In [13]:
cols = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
train_df[cols].corr()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
cohesion,1.000000,0.695459,0.666151,0.690058,0.638689,0.666151
syntax,0.695459,1.000000,0.680562,0.725467,0.709525,0.700025
vocabulary,0.666151,0.680562,1.000000,0.735261,0.654852,0.664292
phraseology,0.690058,0.725467,0.735261,1.000000,0.719746,0.666842
grammar,0.638689,0.709525,0.654852,0.719746,1.000000,0.673301
conventions,0.666151,0.700025,0.664292,0.666842,0.673301,1.000000


In [14]:
cols = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
oof_df[cols].corr()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
cohesion,1.000000,0.947796,0.916701,0.914909,0.828535,0.891541
syntax,0.947796,1.000000,0.940715,0.965292,0.922270,0.931673
vocabulary,0.916701,0.940715,1.000000,0.963708,0.879392,0.896324
phraseology,0.914909,0.965292,0.963708,1.000000,0.945892,0.890577
grammar,0.828535,0.922270,0.879392,0.945892,1.000000,0.858630
conventions,0.891541,0.931673,0.896324,0.890577,0.858630,1.000000
